### 8.1 코너점 검출

In [2]:
import cv2
import numpy as np

def findLocalMaxima(src):
    kernel = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(11, 11))
    
    dilate = cv2.dilate(src, kernel)
    localMax = (src == dilate)
    
    erode = cv2.erode(src, kernel)
    localMax2 = src > erode
    localMax &= localMax2
    points = np.argwhere(localMax==True)
    points[:, [0, 1]] = points[:, [1, 0]]
    return points

src = cv2.imread('c:/data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.preCornerDetect(gray, ksize=3) # corner 찾기
ret, res2 = cv2.threshold(np.abs(res), 0.1, 0, cv2.THRESH_TOZERO) # res가 음수로 나오기에 절대값화 시키고 0.1보다 큰 것들은 0으로
corners = findLocalMaxima(res2)
print(corners.shape)

dst = src.copy()
for x, y in corners:
    cv2.circle(dst, (x, y), 5, (0, 0, 255), 2)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

(8, 2)


### 8.2 고유값과 고유벡터를 이용한 코너점 검출

In [13]:
import cv2
import numpy as np

src = cv2.imread('c:/data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.cornerEigenValsAndVecs(gray, blockSize=5, ksize=3)
print(res.shape)
eigen = cv2.split(res)

T = 0.2
ret, edge = cv2.threshold(eigen[0], T, 255, cv2.THRESH_BINARY)
edge = edge.astype(np.uint8)

corners = np.argwhere(eigen[1] > T)
corners[:, [0, 1]] = corners[:, [1, 0]]
print(len(corners))
print(corners)

dst = src.copy()
for x, y in corners:
    cv2.circle(dst, (x, y), 5, (0, 0, 255), 2)

cv2.imshow('edge', edge)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

(512, 512, 6)
8
[[109 127]
 [264 127]
 [267 167]
 [386 170]
 [109 268]
 [167 271]
 [170 374]
 [386 374]]


### 8.3 최소 고유값으로 corner 검출
* findLocalMaxima() 안 만들어도 corner 좌표 찾아냄

In [22]:
import cv2
import numpy as np

# src = cv2.imread('c:/data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
eigen = cv2.cornerMinEigenVal(gray, blockSize=5)
# _, res = cv2.threshold(eigen, 0.1, 255, cv2.THRESH_BINARY) # corner 점이 좀 더 밝게 보임
print(res.shape)

T = 0.2
corners = np.argwhere(eigen > T)
corners[:, [0, 1]] = corners[:, [1, 0]]
print(len(corners)) # corner의 개수
# print(corners) # 각 corner의 좌표
dst = src.copy()
for x, y in corners:
    cv2.circle(dst, (x, y), 3, (0, 0, 255), 2)

cv2.imshow('dst', dst)
cv2.imshow('eigen', res)
cv2.waitKey()
cv2.destroyAllWindows()

(512, 512)
8


### 8.4 코너점 검출, 일반적으로 corner를 찾을 때 사용

In [15]:
import cv2
import numpy as np

def findLocalMaxima(src):
    kernel = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(11, 11))
    dilate = cv2.dilate(src, kernel)
    localMax = (src == dilate)
    
    erode = cv2.erode(src, kernel)
    localMax2 = src > erode
    localMax &= localMax2
    points = np.argwhere(localMax == True)
    points[:, [0, 1]] = points[:, [1, 0]]
    return points

src = cv2.imread('c:/data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.cornerHarris(gray, blockSize=5, ksize=3, k=0.01)
ret, res = cv2.threshold(np.abs(res), 0.02, 0, cv2.THRESH_TOZERO)
res8 = cv2.normalize(res, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
cv2.imshow('res8', res8)

corners = findLocalMaxima(res)
print(corners)

# 좀 더 정교한 corner 검출
corners = corners.astype(np.float32, order='C')
term_crit = (cv2.TERM_CRITERIA_MAX_ITER+cv2.TERM_CRITERIA_EPS, 10, 0.01)
corners2= cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), term_crit)
print(corners2)

dst = src.copy()

for x, y in np.int32(corners2):
    cv2.circle(dst, (x, y), 3, (0, 0, 255), 2)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

[[109 127]
 [264 127]
 [267 167]
 [386 170]
 [109 268]
 [167 271]
 [170 374]
 [386 374]]
[[107.559364 125.559456]
 [265.44077  125.559235]
 [265.55923  168.44174 ]
 [387.4408   168.55888 ]
 [107.558365 269.44095 ]
 [168.4415   269.55927 ]
 [168.55922  375.4408  ]
 [387.4408   375.4408  ]]


### 8.6 비디오에서 사용하는 corner 검출

In [1]:
import cv2
import numpy as np

src = cv2.imread('c:/data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

K = 5
corners = cv2.goodFeaturesToTrack(gray, K, 0.05, 10)
print(corners)

corners2 = cv2.goodFeaturesToTrack(gray, K, 0.05, 10, useHarrisDetector=True, k=0.04)
print(corners2)

dst = src.copy()
corners = corners.reshape(-1, 2)
for x, y in corners:
    cv2.circle(dst, (int(x), int(y)), 5, (0, 0, 255), -1)

corners2 = corners2.reshape(-1, 2)
for x, y in corners2:
    cv2.circle(dst, (int(x), int(y)), 5, (255, 0, 0), 2)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

[[[387. 375.]]

 [[169. 375.]]

 [[265. 126.]]

 [[168. 270.]]

 [[266. 168.]]]
[[[387. 375.]]

 [[169. 375.]]

 [[265. 126.]]

 [[387. 169.]]

 [[108. 269.]]]


### 비디오 적용한 corner 찾기

In [5]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    K = 50
    corners = cv2.goodFeaturesToTrack(gray, K, 0.05, 10)
    print(corners)

    corners2 = cv2.goodFeaturesToTrack(gray, K, 0.05, 10, useHarrisDetector=True, k=0.04)
    print(corners2)

    dst = frame.copy()
    corners = corners.reshape(-1, 2)
    for x, y in corners:
        cv2.circle(dst, (int(x), int(y)), 5, (0, 0, 255), -1)

    corners2 = corners2.reshape(-1, 2)
    for x, y in corners2:
        cv2.circle(dst, (int(x), int(y)), 5, (255, 0, 0), 2)

    cv2.imshow('dst', dst)
    
    key = cv2.waitKey(25)
    
    if key == 27:
        break

if cap.isOpened():
    cap.release()
cv2.destroyAllWindows()

[[[468. 368.]]

 [[492. 322.]]

 [[144. 468.]]

 [[465. 300.]]

 [[ 96. 352.]]

 [[475. 186.]]

 [[335. 396.]]

 [[153. 424.]]

 [[250. 452.]]

 [[131. 428.]]

 [[479. 438.]]

 [[500. 421.]]

 [[327. 321.]]

 [[566. 294.]]

 [[211. 218.]]

 [[ 85. 352.]]

 [[462. 380.]]

 [[125. 468.]]

 [[308. 437.]]

 [[504. 460.]]

 [[331. 335.]]

 [[534. 130.]]

 [[444. 166.]]

 [[545.  74.]]

 [[ 95. 214.]]

 [[370. 270.]]

 [[134. 462.]]

 [[110. 467.]]

 [[546. 120.]]

 [[598. 297.]]

 [[478. 371.]]

 [[143. 448.]]

 [[456. 179.]]

 [[494. 334.]]

 [[150. 412.]]

 [[534. 403.]]

 [[359. 260.]]

 [[535. 421.]]

 [[120. 423.]]

 [[448. 216.]]

 [[158. 468.]]

 [[188. 216.]]

 [[111. 428.]]

 [[510. 429.]]

 [[513. 440.]]

 [[550. 130.]]

 [[210. 238.]]

 [[635. 410.]]

 [[206. 365.]]

 [[365. 286.]]]
[[[468. 368.]]

 [[493. 321.]]

 [[478. 437.]]

 [[464. 300.]]

 [[335. 396.]]

 [[504. 460.]]

 [[500. 422.]]

 [[144. 468.]]

 [[ 96. 358.]]

 [[475. 186.]]

 [[545.  74.]]

 [[249. 452.]]

 [[308. 

### 8.7 체스보드 패턴 corner 검출

In [7]:
import cv2
import numpy as np

src = cv2.imread('c:/data/chessBoard.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
patternSize = (6, 3)
found, corners = cv2.findChessboardCorners(src, patternSize)
print(corners.shape)

term_crit = (cv2.TERM_CRITERIA_EPS+cv2.TermCriteria_MAX_ITER, 10, 0.01)
corners2 = cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), term_crit)

dst = src.copy()
cv2.drawChessboardCorners(dst, patternSize, corners2, found)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()


(18, 1, 2)


### 8.8 원 패턴 중심점 검출

In [17]:
import cv2
import numpy as np

src = cv2.imread('c:/data/circleGrid.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
patternSize = (6, 4)
found, centers = cv2.findCirclesGrid(src, patternSize)

term_crit = (cv2.TERM_CRITERIA_MAX_ITER+cv2.TERM_CRITERIA_EPS, 10, 0.01)
centers2 = cv2.cornerSubPix(gray, centers, (5, 5), (-1, -1), term_crit)

dst = src.copy()
cv2.drawChessboardCorners(dst, patternSize, centers2, found)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()